In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import time

Using TensorFlow backend.
c:\users\mfili\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mfili\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mfili\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        print(state.shape)
        print(state)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [3]:
env = gym.make('CartPole-v1')
print(env.observation_space.shape)

(4,)


In [ ]:
if __name__ == "__main__":
    start_time = time.time()
    EPISODES = 100
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32
    
    for e in range(EPISODES):
        
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for t in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, t, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")
    elapsed_time = time.time() - start_time
    print(elapsed_time)

(1, 4)
[[-0.03470366  0.00047441  0.01487436  0.03667153]]
(1, 4)
[[-0.03469418  0.19537994  0.01560779 -0.25128152]]
(1, 4)
[[-3.07865770e-02  3.86309699e-05  1.05821632e-02  4.62832604e-02]]
(1, 4)
[[-0.0307858  -0.19523345  0.01150783  0.34228608]]
(1, 4)
[[-0.03469047 -0.39051722  0.01835355  0.63857556]]
(1, 4)
[[-0.04250082 -0.19565593  0.03112506  0.35172849]]
(1, 4)
[[-0.04641394 -0.0009901   0.03815963  0.06902048]]
(1, 4)
[[-0.04643374  0.19356457  0.03954004 -0.21138287]]
(1, 4)
[[-0.04256245  0.38809953  0.03531238 -0.4913355 ]]
(1, 4)
[[-0.03480046  0.58270604  0.02548567 -0.77268359]]
(1, 4)
[[-0.02314634  0.38724287  0.010032   -0.47209208]]
(1, 4)
[[-0.01540148  0.19198068  0.00059016 -0.17626417]]
(1, 4)
[[-0.01156186  0.38709418 -0.00293512 -0.46876086]]
(1, 4)
[[-0.00381998  0.58225747 -0.01231034 -0.76236746]]
(1, 4)
[[ 0.00782517  0.77754681 -0.02755769 -1.05889842]]
(1, 4)
[[ 0.0233761   0.58280055 -0.04873566 -0.77499089]]
(1, 4)
[[ 0.03503212  0.7785578  -0.0642